In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime

# Load datasets
vessel_data = pd.read_csv('../ais_train.csv', sep='|')
schedule_data = pd.read_csv('../schedules_to_may_2024.csv', sep='|')
ports_data = pd.read_csv('../ports.csv', sep='|')
vessels_info = pd.read_csv('../vessels.csv', sep='|')

# Step 1: Convert 'time' column to datetime in vessel_data
vessel_data['time'] = pd.to_datetime(vessel_data['time'])

# Step 2: Get the latest vessel data for each vessel (optional step)
latest_vessel_data = vessel_data.loc[vessel_data.groupby('vesselId')['time'].idxmax()]

# Feature extraction: Create time-based features (hour, day_of_week, month) from the 'time' column
vessel_data['hour'] = vessel_data['time'].dt.hour
vessel_data['day_of_week'] = vessel_data['time'].dt.dayofweek
vessel_data['month'] = vessel_data['time'].dt.month

In [6]:
from geopy.distance import geodesic

# Calculate sailing velocity (distance between positions divided by time difference)
vessel_data['prev_latitude'] = vessel_data['latitude'].shift(1)
vessel_data['prev_longitude'] = vessel_data['longitude'].shift(1)
vessel_data['prev_time'] = vessel_data['time'].shift(1)

In [7]:
# Compute time difference in hours
vessel_data['time_diff'] = (vessel_data['time'] - vessel_data['prev_time']).dt.total_seconds() / 3600

In [8]:
print(vessel_data)

                       time    cog   sog  rot  heading  navstat       etaRaw  \
0       2024-01-01 00:00:25  284.0   0.7    0       88        0  01-09 23:00   
1       2024-01-01 00:00:36  109.6   0.0   -6      347        1  12-29 20:00   
2       2024-01-01 00:01:45  111.0  11.0    0      112        0  01-02 09:00   
3       2024-01-01 00:03:11   96.4   0.0    0      142        1  12-31 20:00   
4       2024-01-01 00:03:51  214.0  19.7    0      215        0  01-25 12:00   
...                     ...    ...   ...  ...      ...      ...          ...   
1522060 2024-05-07 23:59:07  359.1  13.4    0        1        0  05-08 05:00   
1522061 2024-05-07 23:59:08   12.3  17.1    0       13        0  05-10 03:00   
1522062 2024-05-07 23:59:08  269.8  14.9   -1      270        0  05-15 23:00   
1522063 2024-05-07 23:59:08    8.0  18.7    0        6        0  05-08 12:45   
1522064 2024-05-07 23:59:08  336.0  14.3    5      337        0  05-07 23:00   

         latitude  longitude           

In [4]:
# Calculate the distance traveled (in kilometers)
# vessel_data['distance_traveled'] = vessel_data.apply(lambda row: geodesic(
#     (row['prev_latitude'], row['prev_longitude']), (row['latitude'], row['longitude'])
# ).kilometers if pd.notnull(row['prev_latitude']) else np.nan, axis=1)

KeyboardInterrupt: 

In [9]:
# Calculate sailing velocity (km/h)
# vessel_data['sailing_velocity'] = vessel_data['distance_traveled'] / vessel_data['time_diff']

# Merge vessel info into the main dataset
vessel_data = pd.merge(vessel_data, vessels_info[['vesselId', 'CEU', 'length', 'maxSpeed']], on='vesselId', how='left')

# Merge ports data to get port locations
vessel_data = pd.merge(vessel_data, ports_data[['portId', 'latitude', 'longitude']], left_on='portId', right_on='portId', how='left')

In [10]:
# Calculate the distance to the nearest port (or the destination port)
# vessel_data['distance_to_port'] = vessel_data.apply(lambda row: geodesic(
#     (row['latitude'], row['longitude']), (row['latitude_y'], row['longitude_y'])
# ).kilometers if pd.notnull(row['latitude_y']) else np.nan, axis=1)

# Merge schedule data to include planned arrival information
# vessel_data = pd.merge(vessel_data, schedule_data[['vesselId', 'arrivalDate']], on='vesselId', how='left')

# Create time until scheduled arrival feature
vessel_data['time_until_arrival'] = (vessel_data['arrivalDate'] - vessel_data['time']).dt.total_seconds() / 3600  # Time in hours

KeyError: 'latitude'